### BetFair API Guide
https://towardsdatascience.com/an-introduction-to-betfair-api-and-how-to-use-it-e3cdbd79c94b#:~:text=To%20begin%20you%20will%20need,click%20and%20select%20Inspect%20Element.

In [1]:
import json
import datetime
import urllib
import urllib.request
import urllib.error
import requests
import pandas as pd

login using requests.post() to get the SSOID

In [2]:
my_username = "log_runner"
my_password = "mxh.qtz7mub7mdw@KVX"
my_app_key = "6CqiPUMcgYJjnhbh"

payload = 'username=' + my_username + '&password=' + my_password
headers = {'X-Application': my_app_key, 'Content-Type': 'application/x-www-form-urlencoded'}
resp = requests.post('https://identitysso-cert.betfair.com/api/certlogin',data=payload,cert=('./../default_app.crt','./../bf_key_rsa.pem'),headers=headers)
json_resp=resp.json()
SSOID = json_resp['sessionToken']

Here we are setting the URL that will be posting to, the request, and the headers. This particular request will provide a list of all available sports from Betfair. The URL below is set once and is the same URL that all requests are sent to. The event_req is the request that is being sent to the URL. This has to be set up like below bet can be changed when in this format. The method can be changed using the operations that can be found here. req retrieves the HTML response and is then converted into json to allow us to read it.

eventypes is a list of dictionaries

In [3]:
bet_url="https://api.betfair.com/exchange/betting/json-rpc/v1"
event_req = '{"jsonrpc": "2.0", "method": "SportsAPING/v1.0/listEventTypes", "params": {"filter":{ }}, "id": 1}'
headers = {'X-Application': my_app_key, 'X-Authentication': SSOID, 'content-type': 'application/json'}
req = requests.post(bet_url, data=event_req.encode('utf-8'), headers=headers) 
eventTypes = req.json()
eventTypes

{'jsonrpc': '2.0',
 'result': [{'eventType': {'id': '1', 'name': 'Soccer'}, 'marketCount': 3491},
  {'eventType': {'id': '2', 'name': 'Tennis'}, 'marketCount': 2368},
  {'eventType': {'id': '7522', 'name': 'Basketball'}, 'marketCount': 85},
  {'eventType': {'id': '3', 'name': 'Golf'}, 'marketCount': 96},
  {'eventType': {'id': '4', 'name': 'Cricket'}, 'marketCount': 365},
  {'eventType': {'id': '1477', 'name': 'Rugby League'}, 'marketCount': 58},
  {'eventType': {'id': '5', 'name': 'Rugby Union'}, 'marketCount': 34},
  {'eventType': {'id': '6', 'name': 'Boxing'}, 'marketCount': 51},
  {'eventType': {'id': '7', 'name': 'Horse Racing'}, 'marketCount': 871},
  {'eventType': {'id': '8', 'name': 'Motor Sport'}, 'marketCount': 14},
  {'eventType': {'id': '27454571', 'name': 'Esports'}, 'marketCount': 140},
  {'eventType': {'id': '10', 'name': 'Special Bets'}, 'marketCount': 4},
  {'eventType': {'id': '998917', 'name': 'Volleyball'}, 'marketCount': 4},
  {'eventType': {'id': '11', 'name': 'Cy

In [6]:
# example of reading specific event type

print(eventTypes['result'][0]['eventType']['name'])
print(eventTypes['result'][0]['eventType'].keys())

Soccer
dict_keys(['id', 'name'])


In [7]:
Inplay_req = '{"jsonrpc": "2.0", "method": "SportsAPING/v1.0/listMarketTypes", "params": {"filter":{"eventTypeIds":["1"],\
              "inPlayOnly":true}}, "id": 1}'
req = requests.post(bet_url, data=Inplay_req.encode('utf-8'), headers=headers) 
MarketTypes = req.json()
MarketTypes

{'jsonrpc': '2.0',
 'result': [{'marketType': 'HALF_TIME', 'marketCount': 1},
  {'marketType': 'OVER_UNDER_55', 'marketCount': 1},
  {'marketType': 'CORRECT_SCORE', 'marketCount': 1},
  {'marketType': 'ALT_TOTAL_GOALS', 'marketCount': 1},
  {'marketType': 'OVER_UNDER_45', 'marketCount': 1},
  {'marketType': 'HALF_TIME_FULL_TIME', 'marketCount': 1},
  {'marketType': 'OVER_UNDER_85', 'marketCount': 1},
  {'marketType': 'OVER_UNDER_35', 'marketCount': 1},
  {'marketType': 'BOTH_TEAMS_TO_SCORE', 'marketCount': 1},
  {'marketType': 'OVER_UNDER_75', 'marketCount': 1},
  {'marketType': 'ASIAN_HANDICAP', 'marketCount': 1},
  {'marketType': 'DRAW_NO_BET', 'marketCount': 1},
  {'marketType': 'MATCH_ODDS', 'marketCount': 1},
  {'marketType': 'TEAM_B_1', 'marketCount': 1},
  {'marketType': 'DOUBLE_CHANCE', 'marketCount': 1},
  {'marketType': 'TEAM_A_1', 'marketCount': 1},
  {'marketType': 'OVER_UNDER_65', 'marketCount': 1}],
 'id': 1}

In [8]:
MarketType = '["MATCH_ODDS"]'
MarketStartTime= datetime.datetime.now().strftime('%Y-%m-%dT%H:%M:%SZ')
MarketEndTime= (datetime.datetime.now() + datetime.timedelta(hours=24))
MarketEndTime = MarketEndTime.strftime('%Y-%m-%dT%H:%M:%SZ')

user_req='{"jsonrpc": "2.0", "method": "SportsAPING/v1.0/listMarketCatalogue",\
           "params": {"filter":{"eventTypeIds":["1"],"marketTypeCodes":'+MarketType+',\
           "marketStartTime":{"from":"'+MarketStartTime+'", "to":"'+MarketEndTime+'"}},"sort":"FIRST_TO_START",\
           "maxResults":"1", "marketProjection":["RUNNER_METADATA"]}, "id": 1}'

req = urllib.request.Request(bet_url, data=user_req.encode('utf-8'), headers=headers)
response= urllib.request.urlopen(req)
jsonResponse = response.read()
pkg = jsonResponse.decode('utf-8')
market_catalogue = json.loads(pkg) 
market_catalogue['result']

[{'marketId': '1.215229957',
  'marketName': 'Match Odds',
  'totalMatched': 18233.92,
  'runners': [{'selectionId': 55212,
    'runnerName': 'Argentina',
    'handicap': 0.0,
    'sortPriority': 1,
    'metadata': {'runnerId': '55212'}},
   {'selectionId': 15296,
    'runnerName': 'Australia',
    'handicap': 0.0,
    'sortPriority': 2,
    'metadata': {'runnerId': '15296'}},
   {'selectionId': 58805,
    'runnerName': 'The Draw',
    'handicap': 0.0,
    'sortPriority': 3,
    'metadata': {'runnerId': '58805'}}]}]

In [18]:
eventTypeID = '["7"]' #ID for Horse Racing
countryCode= '["GB","IE"]' #Country Codes. Betfair use Alpha-2 Codes under ISO 3166-1
marketTypeCode='["WIN"]' #Market Type
MarketStartTime= datetime.datetime.now().strftime('%Y-%m-%dT%H:%M:%SZ') #Event Start and End times
MarketEndTime = (datetime.datetime.now() + datetime.timedelta(hours=24))
MarketEndTime = MarketEndTime.strftime('%Y-%m-%dT%H:%M:%SZ')
maxResults = str(1000)
sortType = 'FIRST_TO_START' #Sorts the Output
Metadata = 'RUNNER_METADATA' #Provides metadata
inplay = 'false' #still to run

user_req='{"jsonrpc": "2.0", "method": "SportsAPING/v1.0/listMarketCatalogue",\
           "params": {"filter":{"eventTypeIds":'+eventTypeID+',"marketTypeCodes":'+marketTypeCode+',\
           "inPlayOnly":'+inplay+', "marketCountries":'+countryCode+',  \
           "marketStartTime":{"from":"'+MarketStartTime+'", "to":"'+MarketEndTime+'"}},\
           "sort":"'+sortType+'", "maxResults":"'+maxResults+'", "marketProjection":["'+Metadata+'"]}, "id": 1}'

req = urllib.request.Request(bet_url, data=user_req.encode('utf-8'), headers=headers)
response= urllib.request.urlopen(req)
jsonResponse = response.read()
pkg = jsonResponse.decode('utf-8')
result = json.loads(pkg) 
marketCatelogue = result['result']

In [19]:
# explore the results for one runner:

marketCatelogue[0]['runners'][1]

{'selectionId': 42239738,
 'runnerName': 'Tessy Lad',
 'handicap': 0.0,
 'sortPriority': 2,
 'metadata': {'SIRE_NAME': 'Australia',
  'CLOTH_NUMBER_ALPHA': '1',
  'OFFICIAL_RATING': None,
  'COLOURS_DESCRIPTION': 'Royal blue, yellow chevron and armlets, quartered cap',
  'COLOURS_FILENAME': 'c20230615nwb/00847659A.jpg',
  'FORECASTPRICE_DENOMINATOR': '1',
  'DAMSIRE_NAME': 'Medicean',
  'WEIGHT_VALUE': '158',
  'SEX_TYPE': 'G',
  'DAYS_SINCE_LAST_RUN': '162',
  'WEARING': None,
  'OWNER_NAME': 'M&O Construction & Civil Engineering L',
  'DAM_YEAR_BORN': '2011',
  'SIRE_BRED': 'GBR',
  'JOCKEY_NAME': 'Miss Taryn Langley',
  'DAM_BRED': 'GBR',
  'ADJUSTED_RATING': None,
  'runnerId': '42239738',
  'CLOTH_NUMBER': '1',
  'SIRE_YEAR_BORN': '2011',
  'TRAINER_NAME': 'R Hughes',
  'COLOUR_TYPE': 'b',
  'AGE': '4',
  'DAMSIRE_BRED': 'GBR',
  'JOCKEY_CLAIM': '5',
  'FORM': '21621-7',
  'FORECASTPRICE_NUMERATOR': '8',
  'BRED': None,
  'DAM_NAME': 'Maracuja',
  'DAMSIRE_YEAR_BORN': '1997',
  'S

In [20]:
priceProjection = '["EX_BEST_OFFERS"]'
marketId = str(marketCatelogue[0]['marketId'])
selectionId = str(marketCatelogue[0]['runners'][0]['selectionId'])

price_req = '{"jsonrpc": "2.0", "method": "SportsAPING/v1.0/listRunnerBook", "params": {"locale":"en",\
            "marketId":"'+marketId+'", "selectionId":"'+selectionId+'", '\
            '"priceProjection":{"priceData":'+priceProjection+'},"orderProjection":"ALL"},"id":1}'
req = urllib.request.Request(bet_url, data=price_req.encode('utf-8'), headers=headers)
price_response= urllib.request.urlopen(req)
price_jsonResponse = price_response.read()
price_pkg = price_jsonResponse.decode('utf-8')
price_result = json.loads(price_pkg)
price_result['result'][0]

{'marketId': '1.215211389',
 'isMarketDataDelayed': True,
 'status': 'OPEN',
 'betDelay': 0,
 'bspReconciled': False,
 'complete': True,
 'inplay': False,
 'numberOfWinners': 1,
 'numberOfRunners': 13,
 'numberOfActiveRunners': 12,
 'lastMatchTime': '2023-06-15T10:57:34.955Z',
 'totalMatched': 13124.34,
 'totalAvailable': 683501.47,
 'crossMatching': True,
 'runnersVoidable': False,
 'version': 5295932811,
 'runners': [{'selectionId': 37065275,
   'handicap': 0.0,
   'status': 'ACTIVE',
   'adjustmentFactor': 16.277,
   'lastPriceTraded': 5.9,
   'totalMatched': 0.0,
   'ex': {'availableToBack': [{'price': 5.7, 'size': 13.86},
     {'price': 5.6, 'size': 18.34},
     {'price': 5.5, 'size': 16.72}],
    'availableToLay': [{'price': 5.9, 'size': 14.34},
     {'price': 6.0, 'size': 68.6},
     {'price': 6.2, 'size': 68.7}],
    'tradedVolume': []}}]}

In [34]:
def BestHorseForm():
    
    Rating = float(0)
    Index = float(0)
    FormRatingAvg = float(100)
    FormRatingList = []
    horsename = []
    selectionID=[]
    
    eventTypeID = '["7"]' #ID for Horse Racing
    countryCode= '["GB","IE"]' #Country Codes. Betfair use Alpha-2 Codes under ISO 3166-1
    marketTypeCode='["WIN"]' #Market Type
    MarketStartTime= datetime.datetime.now().strftime('%Y-%m-%dT%H:%M:%SZ') #Event Start and End times
    MarketEndTime = (datetime.datetime.now() + datetime.timedelta(hours=24))
    MarketEndTime = MarketEndTime.strftime('%Y-%m-%dT%H:%M:%SZ')
    maxResults = str(1000)
    sortType = 'FIRST_TO_START' #Sorts the Output
    Metadata = 'RUNNER_METADATA' #Provides metadata
    inplay = 'false' #still to run
    priceProjection = '["EX_BEST_OFFERS"]'#Best odds
    
    #Create an empty dataframe
    d = {'Horse Name': [], 'Form':[], 'Race': [], 'Odds':[]}
    Results = pd.DataFrame(data=d)
    
    my_username = "log_runner"
    my_password = "mxh.qtz7mub7mdw@KVX"
    my_app_key = "6CqiPUMcgYJjnhbh"
    my_cert = './../default_app.crt'
    my_key = './../bf_key_rsa.pem'

    payload = 'username=' + my_username + '&password=' + my_password
    headers = {'X-Application': my_app_key, 'Content-Type': 'application/x-www-form-urlencoded'}
    bet_url="https://api.betfair.com/exchange/betting/json-rpc/v1"
    resp = requests.post('https://identitysso-cert.betfair.com/api/certlogin',data=payload,cert=(my_cert,my_key),headers=headers)
    resp_json=resp.json()
    SSOID = resp_json['sessionToken']
    
    headers = {'X-Application': my_app_key, 'X-Authentication': SSOID, 'content-type': 'application/json'}
    
    user_req='{"jsonrpc": "2.0", "method": "SportsAPING/v1.0/listMarketCatalogue",\
           "params": {"filter":{"eventTypeIds":'+eventTypeID+',"marketTypeCodes":'+marketTypeCode+',\
           "inPlayOnly":'+inplay+', "marketCountries":'+countryCode+',\
           "marketStartTime":{"from":"'+MarketStartTime+'", "to":"'+MarketEndTime+'"}},\
           "sort":"'+sortType+'", "maxResults":"'+maxResults+'", "marketProjection":["'+Metadata+'"]}, "id": 1}'

    req = urllib.request.Request(bet_url, data=user_req.encode('utf-8'), headers=headers)
    response= urllib.request.urlopen(req)
    jsonResponse = response.read()
    pkg = jsonResponse.decode('utf-8')
    result = json.loads(pkg) 
    marketCatelogue = result['result']
    

    for x in range(len(marketCatelogue)):
        for w in range(len(marketCatelogue[x]['runners'])):
            runnerform = marketCatelogue[x]['runners'][w]['metadata']['FORM']
            if runnerform is None:
                runnerform = 'e'
            
            runnerformList = list(runnerform)
            
            for Entry in runnerformList:
                if Entry == 'R':#refusal to jump hurdle
                    Rating = float(Rating) + float(5)
                    Index = Index + 1
                elif Entry == 'e':#First Race
                    Rating = float(Rating) + float(10)
                    Index = Index + 1
                elif Entry == '0':#finished higher than 9th
                    Rating = float(Rating) + float(10)
                    Index = Index + 1
                elif Entry == 'F':#fell
                    Rating = float(Rating) + float(5)
                    Index = Index + 1
                elif Entry == 'U':#unseated rider
                    Rating = float(Rating) + float(3)
                    Index = Index + 1
                elif Entry == 'x':#horse has not started in a race for 3 months or more
                    Rating = float(Rating) + float(3)
                    Index = Index + 1
                elif Entry == 'C':#horse has won before at this same race distance and at this same track.
                    Rating = float(Rating) + float(.5)
                    Index = Index + 1
                elif Entry == 'B':#horse started favorite at it's last start, but it did not win
                    Rating = float(Rating) + float(3.5)
                    Index = Index + 1
                elif Entry == '/':#represents two seasons ago
                    Rating = float(Rating) + float(8)
                    Index = Index + 1
                elif Entry == '-':#represents one season ago
                    Rating = float(Rating) + float(4)
                    Index = Index + 1
                elif Entry == 'P':#pulled up by jockey
                    Rating = float(Rating) + float(4)
                    Index = Index + 1
                else:
                    try:
                        Rating = float(Rating) + float(Entry)
                    except ValueError:
                        Rating = float(Rating)
                    Index = Index + 1
                                                                
            rating = float(Rating)/float(Index)
            FormRatingList.append(rating)
                
        for t in range(len(FormRatingList)):
            if FormRatingList[t] < FormRatingAvg:
                FormRatingAvg = FormRatingList[t]
                horsename.append(marketCatelogue[x]['runners'][t]['runnerName'])
                selectionID.append(marketCatelogue[x]['runners'][t]['selectionId'])

        price_req = '{"jsonrpc": "2.0", "method": "SportsAPING/v1.0/listRunnerBook", "params": {"locale":"en", \
                    "marketId":"'+str(marketCatelogue[x]['marketId'])+'",\
                    "selectionId":"'+str(selectionID[-1])+'",\
                    "priceProjection":{"priceData":'+priceProjection+'},"orderProjection":"ALL"},"id":1}'
       
        req = urllib.request.Request(bet_url, data=price_req.encode('utf-8'), headers=headers)
        price_response= urllib.request.urlopen(req)
        price_jsonResponse = price_response.read()
        price_pkg = price_jsonResponse.decode('utf-8')
        price_result = json.loads(price_pkg) 
        
        horse_name = str(horsename[-1])
        horse_rating = str(int(FormRatingAvg))
        race_name = str(marketCatelogue[x]['marketName'])
        try:
            horse_price = str(price_result['result'][0]['runners'][0]['ex']['availableToBack'][0]['price'])
        except IndexError:
            horse_price = '-1'
        
        new_row = {'Horse Name':horse_name, 'Form':horse_rating, 'Race':race_name, 'Odds':horse_price}
        # Results = Results.append({'Horse Name':str(horsename[-1]), 'Form':str(int(FormRatingAvg)), 'Race':str(marketCatelogue[x]['marketName']), 'Odds':str(price_result['result'][0]['runners'][0]['ex']['availableToBack'][0]['price'])}, ignore_index=True)
        # Results = Results.append(new_data, ignore_index=True)
        Results.loc[len(Results)] = new_row
        
        Rating = float(0)
        Index = float(0)
        FormRatingAvg = float(100)
        FormRatingList.clear()
        horsename.clear()
        selectionID.clear
                 
    return Results

In [35]:
BestHorseForm()

,Horse Name,Form,Race,Odds
0,Mabre,4,1m2f Hcap,8.4
1,Book Of Tales,3,1m3f Hcap,2.1
2,Moyet,3,6f Nov Stks,2.44
3,Overlooked,8,6f Nov Stks,19.5
4,Hidden Pearl,3,1m6f Hcap,4.2
5,Master Chartwell,5,1m Mdn Stks,3.55
6,Sea Of Thieves,3,1m Nov Stks,8.4
7,Beau Roc,3,6f Mdn Stks,4.8
8,Coloane,3,1m Hcap,4.3
9,Monopolise,3,1m Hcap,4.5
